In [1]:
import pandas as pd

In [2]:
sales = pd.read_csv("../DataSet/start_vgsales.csv")
steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]
steam.columns = ["player_id", "game_title", "type", "value"]

C:\Users\david\AppData\Local\Temp\ipykernel_16428\560633259.py:2: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]


In [3]:
steam = steam[steam["type"] == "purchase"]

In [4]:
df = steam.merge(sales, how='inner',left_on="game_title", right_on="Name")
df.head()

,player_id,game_title,type,value,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,151603712,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
1,151603712,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09
2,151603712,Fallout 4,purchase,1.0,1632,Fallout 4,PC,2015.0,Role-Playing,Bethesda Softworks,0.50,0.63,0.00,0.10,1.23
3,87445402,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
4,87445402,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09


In [5]:
df = pd.read_csv('test2.csv', index_col="player_id").reset_index().drop_duplicates()
#df = df.pivot_table(values="value", columns="game_title", index="player_id").fillna(0)
#df = df.apply(lambda x: x != 0, axis=0).melt(ignore_index=False).reset_index()
#df["value"] = df["value"].replace({False: 0, True: 1})
# df = df.melt(ignore_index=False).reset_index()
# df = df[df["value"] != 0]
df["value"] = pd.qcut(x=df["value"],q=5, labels=[1,2,3,4,5])
df = df.rename(columns={"game_title": "Genre"})
df

,player_id,Genre,value
0,151603712,Fallout 4,5
3,87445402,Fallout 4,5
6,25096601,Fallout 4,2
9,211925330,Fallout 4,5
12,115396529,Fallout 4,4
...,...,...,...
35504,136137921,Tom Clancy's Ghost Recon Advanced Warfighter 2,1
35507,174838943,Imperial Glory,5
35508,124851136,Jack Keane,2
35509,20772968,BloodRayne,3


In [6]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
users = df
users["player_id"] = np.asarray(users["player_id"]).astype(np.int32)
# users[["value"]] = scaler.fit_transform(users[["value"]])
unique_user_ids = users["player_id"].unique()
users

,player_id,Genre,value
0,151603712,Fallout 4,5
3,87445402,Fallout 4,5
6,25096601,Fallout 4,2
9,211925330,Fallout 4,5
12,115396529,Fallout 4,4
...,...,...,...
35504,136137921,Tom Clancy's Ghost Recon Advanced Warfighter 2,1
35507,174838943,Imperial Glory,5
35508,124851136,Jack Keane,2
35509,20772968,BloodRayne,3


In [8]:
genres = users["Genre"].unique()
unique_genres = genres
genres

array(['Fallout 4', 'Spore', 'Left 4 Dead 2', 'Left 4 Dead',
       'Tomb Raider', 'BioShock Infinite', 'Grand Theft Auto IV',
       'BioShock', 'Ultra Street Fighter IV',
       "Sid Meier's Civilization V", 'L.A. Noire', '7 Days to Die',
       'Company of Heroes', "Assassin's Creed II", 'Grand Theft Auto V',
       'Far Cry 3', 'Rocket League', 'Far Cry 4', 'Dying Light',
       'Just Cause 2', "Mirror's Edge", 'Portal 2', 'Tropico 4',
       'F.E.A.R.', 'Sleeping Dogs', 'Saints Row IV', 'Half-Life 2',
       'The Talos Principle', 'DiRT 3', 'Metro 2033',
       "Assassin's Creed III", 'Sniper Elite 3', 'Dishonored',
       'Dead Space', 'The Binding of Isaac', 'LEGO The Lord of the Rings',
       'Goat Simulator', 'Mass Effect', 'DiRT Showdown', 'Borderlands 2',
       'Trine', 'Far Cry 2', 'Oil Rush', 'Psychonauts', 'Dead Island',
       'GRID 2', 'Sonic & All-Stars Racing Transformed',
       'Prison Architect', 'Valkyria Chronicles', 'The Crew',
       'Sonic Generations', 'Dun

In [9]:
tf.random.set_seed(42)
users_tensor = tf.data.Dataset.from_tensor_slices(dict(users))
users_shuffled = users_tensor.shuffle(seed=42, buffer_size=100_000, reshuffle_each_iteration=False)

users_train = users_shuffled.take(8_000)
users_test = users_shuffled.skip(8_000).take(958)
print(len(users_test), users_test)

958 <TakeDataset element_spec={'player_id': TensorSpec(shape=(), dtype=tf.int32, name=None), 'Genre': TensorSpec(shape=(), dtype=tf.string, name=None), 'value': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [10]:
genres_tensor = tf.data.Dataset.from_tensor_slices(genres)

In [11]:
for r in genres_tensor.take(5):
    print(r)

tf.Tensor(b'Fallout 4', shape=(), dtype=string)
tf.Tensor(b'Spore', shape=(), dtype=string)
tf.Tensor(b'Left 4 Dead 2', shape=(), dtype=string)
tf.Tensor(b'Left 4 Dead', shape=(), dtype=string)
tf.Tensor(b'Tomb Raider', shape=(), dtype=string)


In [12]:
from typing import Dict, Text

In [13]:
class MultiTaskModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.
  def __init__(
          self,
          user_model: tf.keras.Model,
          genre_model: tf.keras.Model,
          retrieval_task: tfrs.tasks.Retrieval,
          ranking_task: tfrs.tasks.Ranking,
          rating_weight:float = .5,
          retrieval_weight:float = .5,
  ):
    super().__init__()

      # Set up user and movie representations.
    self.user_model = user_model
    self.genre_model = genre_model

    # Compute predictions.
    self.rating_model = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
    ])

    
    self.rating_task: tf.keras.layers.Layer = ranking_task
    self.retrieval_task: tf.keras.layers.Layer = retrieval_task

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight


  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["player_id"])
    # And pick out the movie features and pass them into the movie model.
    genre_embeddings = self.genre_model(features["Genre"])

    return (
        user_embeddings,
        genre_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, genre_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("value")

    user_embeddings, genre_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, genre_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)




In [14]:
user_model = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(mask_token=None, vocabulary=unique_user_ids),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 64)
])
genre_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(mask_token=None, vocabulary=unique_genres),
    tf.keras.layers.Embedding(len(unique_genres) + 1, 64)
])

In [15]:
ranking_task = tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

retrieval_task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=genres_tensor.batch(128).map(genre_model)
    )
)

In [16]:
model = MultiTaskModel(
    genre_model=genre_model, 
    user_model=user_model, 
    retrieval_task=retrieval_task, 
    ranking_task=ranking_task,
    rating_weight=.5,
    retrieval_weight=.5,
)
model.compile(optimizer=tf.keras.optimizers.Adagrad(.01))

In [17]:
cached_train = users_train.shuffle(100_000).batch(8192).cache()
cached_test = users_test.batch(4096).cache()

In [18]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_train, return_dict=True)

print(f"Retrieval top-5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")


Epoch 1/3
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1/1 [==============================] - 16s 16s/step - root_mean_squared_error: 3.3280 - factorized_top_k/top_1_categorical_accuracy: 0.0016 - factorized_top_k/top_5_categorical_accuracy: 0.0111 - factorized_top_k/top_10_categorical_accuracy: 0.0264 - factorized_top_k/top_50_categorical_accuracy: 0.1334 - factorized_top_k/top_100_categorical_accuracy: 0.2651 - loss: 35954.1953 - regularization_loss: 0.0000e+00 - total_loss: 35954.1953
Epoch 2/3
1/1 [==============================] - 4s 4s/step - root_mean_squared_error: 3.2686 - factorized_top_k/top_1_categorical_accuracy: 0.0034 - factorized_top_k/top_5_categorical_accuracy: 0.0213 - factorized_top_k/top_10_categorical_accuracy: 0.0439 - factorized_top_k/top_50_categorical_accuracy: 0.1813 - factorized_top_k/top_100_categorical_

In [19]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 431ms/step - root_mean_squared_error: 3.1100 - factorized_top_k/top_1_categorical_accuracy: 0.0021 - factorized_top_k/top_5_categorical_accuracy: 0.0157 - factorized_top_k/top_10_categorical_accuracy: 0.0324 - factorized_top_k/top_50_categorical_accuracy: 0.1493 - factorized_top_k/top_100_categorical_accuracy: 0.2662 - loss: 3293.3220 - regularization_loss: 0.0000e+00 - total_loss: 3293.3220


{'root_mean_squared_error': 3.109992742538452,
 'factorized_top_k/top_1_categorical_accuracy': 0.002087682718411088,
 'factorized_top_k/top_5_categorical_accuracy': 0.015657620504498482,
 'factorized_top_k/top_10_categorical_accuracy': 0.032359082251787186,
 'factorized_top_k/top_50_categorical_accuracy': 0.14926931262016296,
 'factorized_top_k/top_100_categorical_accuracy': 0.26617953181266785,
 'loss': 3293.322021484375,
 'regularization_loss': 0,
 'total_loss': 3293.322021484375}

In [23]:
res = []
for genre in unique_genres:
  trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
        "player_id": np.array([43607599]),
        "Genre": np.array([genre])
    })
  res.append([genre, predicted_rating.numpy()[0][0]])
res = pd.DataFrame(data=res, columns = ["Genre", "Rating"])
res.sort_values(by="Rating", ascending=False)

,Genre,Rating
302,Hospital Tycoon,0.229692
41,Far Cry 2,0.223948
2,Left 4 Dead 2,0.223851
43,Psychonauts,0.222881
286,Contrast,0.222101
...,...,...
324,Elven Legacy,0.153471
50,Sonic Generations,0.153152
292,Dark Void,0.150749
338,Gray Matter,0.150611
